In [2]:
# Step 1: Install all necessary packages
!pip install flask pyngrok diffusers transformers accelerate safetensors

# Step 2: Import libraries
import os
import io
import threading
from flask import Flask, request, send_file
from pyngrok import ngrok
from diffusers import StableDiffusionPipeline
import torch
from huggingface_hub import login
from google.colab import userdata

# --- Login to ngrok ---
try:
    NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    print("ngrok authentication successful.")
except Exception as e:
    print(f"ngrok authentication failed: {e}")

# --- Login to Hugging Face ---
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("Hugging Face login successful.")
except Exception as e:
    print(f"Hugging Face login failed: {e}")

# Step 3: Load the more memory-efficient v1.5 Model
print("Loading Stable Diffusion v1.5 model (Realistic Vision v5.1)...")
model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
print("Model loaded successfully.")

# Step 4: Create the Flask App
app = Flask(__name__)

@app.route('/generate-image', methods=['POST'])
def generate_image_endpoint():
    data = request.get_json()
    prompt = data.get('prompt')
    negative_prompt = data.get('negative_prompt', 'cartoon, 3d, (worst quality, low quality, normal quality:2)')

    if not prompt:
        return "Prompt is required", 400

    print(f"Generating image for prompt: {prompt}")

    image = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=25,
        guidance_scale=7.5
    ).images[0]

    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)

    print("Image generated. Sending response.")
    return send_file(img_byte_arr, mimetype='image/png')

# Step 5: Start the Flask app and ngrok tunnel
def run_app():
  app.run(port=5000)

thread = threading.Thread(target=run_app)
thread.start()

public_url = ngrok.connect(5000)
print("------------------------------------------------------------")
print(f"AI ENGINE IS READY!")
print(f"Copy this URL into your main website's .env file:")
print(f"AI_API_ENDPOINT={public_url}/generate-image")
print("------------------------------------------------------------")

ngrok authentication successful.
Hugging Face login successful.
Loading Stable Diffusion v1.5 model (Realistic Vision v5.1)...


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Model loaded successfully.
 * Serving Flask app '__main__'
 * Debug mode: off
------------------------------------------------------------
AI ENGINE IS READY!
Copy this URL into your main website's .env file:
AI_API_ENDPOINT=NgrokTunnel: "https://b0f25e720f6b.ngrok-free.app" -> "http://localhost:5000"/generate-image
------------------------------------------------------------


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
